In [ ]:
import pandas as pd
import numpy as np
from gensim import models
ko_model = models.fasttext.load_facebook_model('data/cc.ko.300.bin.gz')


: 

In [366]:
df_job_info = pd.read_csv('career_map_visualization/data_analysis/job_data.csv', encoding='utf-8')
df_job_major_subject = pd.read_csv('career_map_visualization/data_analysis/job_major_subject.csv', encoding='utf-8')
df_major_info = pd.read_csv('career_map_visualization/data_analysis/major_info.csv', encoding='utf-8')
df_subject_info = pd.read_csv('subject_description_2015.csv', encoding='utf-8',header = 0, names = (['index', 'subject_name', 'description', 'ability', 'related_subjects','type']))

#select rows where job is 통계학연구원
job_name = '통계학연구원'
job_desc = df_job_info[df_job_info['job'] == job_name].job_summary.values[0] 

def similarity_function(method ='fasttext', *args):
    # sourcery skip: merge-comparisons, merge-duplicate-blocks, remove-redundant-if
    if method == 'fasttext':
        return ko_model.wv.similarity(*args)
    elif method == 'SBERT':
        return ko_model.wv.similarity(*args)
    else:
        print('No method')
    
def job_major_subject_matching(job_name):

    # job-major matching
    majorlist = list(df_job_major_subject[df_job_major_subject['job'] == job_name].major.unique())
    job_major = {job_name :majorlist}

    #major-subject matching
    subjects = df_job_major_subject[df_job_major_subject['job']==job_name]
    subject_details = subjects.groupby('major', group_keys=False).subject_details.apply(lambda x: ','.join(x))

    major_subject = dict()
    job_subject = dict()

    for idx, line in enumerate(subject_details):
        tmp = line.replace(':', ',')
        tmp = tmp.split(',')
        # '~교과'로 되어 있는 노이즈 데이터 제거
        for e in tmp:
            if '교과' in e:
                tmp.remove(e)
        tmp = [i.replace(' ','') for i in tmp]

        # '수학II'가 아니라 그냥 'II'라고 되어 있는 과목명을 바로 직전의 과목명을 참고하여 '수학II'로 변경
        for e in tmp:
            if len(e) == 1:
                i = tmp.index(e)
                tmp[i] = tmp[i-1][:-1]+e

        #Add key and value to dictionarry
        major_subject[subject_details.keys()[idx]] = tmp

        subject_set = set()
        for value in major_subject.values():
            for v in value:
                subject_set.add(v)

        job_subject = {job_name : list(subject_set)}

    return job_major, major_subject, job_subject
#from job_major, get major list and calculate similarity between job and major
def get_job_major_similarity(df_job_info, df_major_info, job_major, sim_method):
    job_major_sim = dict()
    for key, value in job_major.items():
        for v in value:
        #get major description
            major_desc = df_major_info[df_major_info['major'] == v].major_summary.values[0]
        #get job description
            job_desc = df_job_info[df_job_info['job'] == key].job_summary.values[0]
        #calculate similarity between job description and major description
            job_major_sim[v] = similarity_function(sim_method, major_desc, job_desc)
    return job_major_sim
#from major_subject, get subject list and calculate similarity between major description and subject description
def get_major_subject_similarity(df_major_info, df_subject_info, major_subject, sim_method):
    major_subject_sim = dict()

    for key, value in major_subject.items():
        one_major_sim = dict()
        for v in value:
        #get major description
            major_desc = df_major_info[df_major_info['major'] == key].major_summary.values[0]
        #get subject description(try catch)
            try:
                tmp_df_subject_info = df_subject_info.replace(' ', '', regex=True)
                subject_desc = df_subject_info[tmp_df_subject_info['subject_name']== v].description.values[0]
            #calculate similarity between major description and subject description
                one_major_sim[v] = similarity_function(sim_method, major_desc, subject_desc)
            except Exception:
                continue
        major_subject_sim[key] = one_major_sim
    return major_subject_sim

def get_job_subject_similarity_1(df_job_info, df_subject_info, job_subject, sim_method):
    job_subject_sim = dict()
    for key, value in job_subject.items():
        for v in value:
        #get major description
            job_desc = df_job_info[df_job_info['job'] == key].job_summary.values[0]
        #get subject description(try catch)
            try:
                tmp_df_subject_info = df_subject_info.replace(' ', '', regex=True)
                subject_desc = df_subject_info[tmp_df_subject_info['subject_name']== v].description.values[0]
            #calculate similarity between major description and subject description
                job_subject_sim[v] = similarity_function(sim_method, job_desc, subject_desc)
            except Exception:
                continue
    return job_subject_sim

def get_job_subject_similarity_2(job_major_sim, major_subject_sim):
    job_subject_sim_2 = dict()
    for major, majorsim in job_major_sim.items():
        for subject, subjectsim in major_subject_sim.get(major).items():
            try:
                job_subject_sim_2[subject] += majorsim*subjectsim
            except Exception:
                job_subject_sim_2[subject] = majorsim*subjectsim
    return job_subject_sim_2

def get_subject_subject_similarity(job_subject_sim, df_subject_info, threshold, sim_method):
    subject_list = list(job_subject_sim.keys())
    num_subject = len(subject_list)
    subject_dict = dict()

    tmp_df_subject_info = df_subject_info.replace(' ', '', regex=True)

    for idx1, subject1 in enumerate(subject_list):
        subject1_desc = df_subject_info[tmp_df_subject_info['subject_name']== subject1].description.values[0]
        for idx2, subject2 in enumerate(subject_list):
            if idx1 >= idx2: continue
            subject2_desc = df_subject_info[tmp_df_subject_info['subject_name']== subject2].description.values[0]
            similarity = similarity_function(sim_method, subject1_desc, subject2_desc)
            if similarity > threshold:
                subject_dict[(subject1, subject2)] = similarity    
    return subject_dict  


def similarity_all(job_name):
    job_major, major_subject, job_subject = job_major_subject_matching(job_name)
    job_major_sim = get_job_major_similarity(df_job_info, df_major_info, job_major, sim_method="fasttext")
    major_subject_sim = get_major_subject_similarity(df_major_info, df_subject_info, major_subject, sim_method="fasttext")
    job_subject_sim_1 = get_job_subject_similarity_1(df_job_info, df_subject_info, job_subject, sim_method="fasttext")
    job_subject_sim_2 = get_job_subject_similarity_2(job_major_sim, major_subject_sim)
    subject_subject_sim = get_subject_subject_similarity(job_subject_sim_1, df_subject_info, threshold=0.98, sim_method="fasttext")
    
    return job_major_sim, major_subject_sim, job_subject_sim_1, job_subject_sim_2, subject_subject_sim

job_major_sim, major_subject_sim, job_subject_sim_1, job_subject_sim_2, subject_subject_sim = similarity_all('통계학연구원')


In [367]:
subject_subject_sim

{('국제관계와국제기구', '미적분'): 0.98304105,
 ('국제관계와국제기구', '기하'): 0.98223466,
 ('국제관계와국제기구', '사회문제탐구'): 0.9812708,
 ('국제관계와국제기구', '수학Ⅱ'): 0.98201454,
 ('국제관계와국제기구', '수학Ⅰ'): 0.981706,
 ('국제관계와국제기구', '확률과통계'): 0.9825937,
 ('국제관계와국제기구', '국제법'): 0.984058,
 ('미적분', '경제수학'): 0.98891395,
 ('미적분', '기하'): 0.9956279,
 ('미적분', '실용수학'): 0.98939145,
 ('미적분', '심화수학Ⅰ'): 0.98413765,
 ('미적분', '수학과제탐구'): 0.992306,
 ('미적분', '사회문제탐구'): 0.9833946,
 ('미적분', '수학Ⅱ'): 0.99700487,
 ('미적분', '수학Ⅰ'): 0.9967511,
 ('미적분', '확률과통계'): 0.9966041,
 ('미적분', '고급수학Ⅱ'): 0.98296213,
 ('미적분', '국제법'): 0.9802181,
 ('미적분', '심화수학Ⅱ'): 0.9874341,
 ('경제수학', '기하'): 0.9899689,
 ('경제수학', '실용수학'): 0.984996,
 ('경제수학', '수학과제탐구'): 0.98543346,
 ('경제수학', '수학Ⅱ'): 0.9873973,
 ('경제수학', '수학Ⅰ'): 0.9881711,
 ('경제수학', '확률과통계'): 0.98746634,
 ('경제수학', '심화수학Ⅱ'): 0.98124003,
 ('기하', '실용수학'): 0.9884886,
 ('기하', '심화수학Ⅰ'): 0.98318005,
 ('기하', '수학과제탐구'): 0.9922701,
 ('기하', '사회문제탐구'): 0.9828272,
 ('기하', '수학Ⅱ'): 0.9950183,
 ('기하', '수학Ⅰ'): 0.99509025,
 ('기하', '확률과통계'):

In [356]:
import numpy as np
job_major, major_subject, job_subject = job_major_subject_matching(job_name)
sim_method = 'fasttext'
treshold = 0.5

subject_list = list(job_subject_sim.keys())
num_subject = len(subject_list)
subject_dict = dict()

tmp_df_subject_info = df_subject_info.replace(' ', '', regex=True)

for idx1, subject1 in enumerate(subject_list):
    subject1_desc = df_subject_info[tmp_df_subject_info['subject_name']== subject1].description.values[0]
    for idx2, subject2 in enumerate(subject_list):
        if idx1 >= idx2: continue
        subject2_desc = df_subject_info[tmp_df_subject_info['subject_name']== subject2].description.values[0]
        similarity = similarity_function(sim_method, subject1_desc, subject2_desc)
        if similarity > treshold:
            subject_dict[(subject1, subject2)] = similarity         

In [358]:
subject_dict

{('국제관계와국제기구', '미적분'): 0.98304105,
 ('국제관계와국제기구', '국제경제'): 0.9756322,
 ('국제관계와국제기구', '정보과학'): 0.96314317,
 ('국제관계와국제기구', '경제수학'): 0.9769358,
 ('국제관계와국제기구', '기하'): 0.98223466,
 ('국제관계와국제기구', '지역이해'): 0.95388067,
 ('국제관계와국제기구', '실용수학'): 0.9774312,
 ('국제관계와국제기구', '심화수학Ⅰ'): 0.9780521,
 ('국제관계와국제기구', '논리학'): 0.9680717,
 ('국제관계와국제기구', '물리학Ⅱ'): 0.9604839,
 ('국제관계와국제기구', '실용경제'): 0.9728459,
 ('국제관계와국제기구', '수학과제탐구'): 0.9796411,
 ('국제관계와국제기구', '사회문제탐구'): 0.9812708,
 ('국제관계와국제기구', '생활과윤리'): 0.958219,
 ('국제관계와국제기구', '수학Ⅱ'): 0.98201454,
 ('국제관계와국제기구', '교육학'): 0.9624548,
 ('국제관계와국제기구', '경제'): 0.9696143,
 ('국제관계와국제기구', '철학'): 0.9672786,
 ('국제관계와국제기구', '수학Ⅰ'): 0.981706,
 ('국제관계와국제기구', '확률과통계'): 0.9825937,
 ('국제관계와국제기구', '고급수학Ⅱ'): 0.9751877,
 ('국제관계와국제기구', '세계문제와미래사회'): 0.9645006,
 ('국제관계와국제기구', '정보'): 0.9680779,
 ('국제관계와국제기구', '국제법'): 0.984058,
 ('국제관계와국제기구', '고급수학Ⅰ'): 0.96052706,
 ('국제관계와국제기구', '논술'): 0.965646,
 ('국제관계와국제기구', '심화수학Ⅱ'): 0.9794658,
 ('미적분', '국제경제'): 0.97363436,
 ('미적분', '정보과학'): 0.9703

In [338]:
list(job_subject_sim.keys())

['국제관계와국제기구',
 '미적분',
 '국제경제',
 '정보과학',
 '경제수학',
 '기하',
 '지역이해',
 '실용수학',
 '심화수학Ⅰ',
 '논리학',
 '물리학Ⅱ',
 '실용경제',
 '수학과제탐구',
 '사회문제탐구',
 '생활과윤리',
 '수학Ⅱ',
 '교육학',
 '경제',
 '철학',
 '수학Ⅰ',
 '확률과통계',
 '고급수학Ⅱ',
 '세계문제와미래사회',
 '정보',
 '국제법',
 '고급수학Ⅰ',
 '논술',
 '심화수학Ⅱ']

In [331]:
similarity_array[0]

array([ 1.00000012,  0.07343709,  0.04949116,  0.23650432,  0.29545674,
        0.22307117,  0.18630242, -0.02356629,  0.60338449,  0.69706404,
        0.51952612,  0.51793867,  0.40715522,  0.22714816,  0.19165629,
        0.24652296,  0.24698842,  0.40658692,  0.67522371,  0.61318409,
        0.2089788 ,  0.43785614,  0.4005399 ,  0.13709059,  0.2528331 ,
        0.12236193,  0.14915164,  0.04320832,  0.10781867,  0.09778093,
        0.25257027,  0.21666238,  0.4886921 ,  0.41406667,  0.0616535 ,
        0.05119729,  0.25250509,  0.3853488 ,  0.623496  ,  0.24679399])

In [328]:
similarity_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [267]:
job_major_sim, major_subject_sim, job_subject_sim = similarity_all("통계학연구원")
pd.DataFrame.from_dict((job_subject_sim), orient='index', columns=['similarity']).sort_values(by=['similarity'], ascending=False)

,similarity
사회문제탐구,0.984253
미적분,0.983122
확률과통계,0.982581
심화수학Ⅱ,0.982466
수학Ⅰ,0.982430
수학과제탐구,0.982322
수학Ⅱ,0.982241
심화수학Ⅰ,0.981749
기하,0.981438
실용수학,0.981082


In [266]:
job_subject_sim

{'국제관계와국제기구': 0.9772885,
 '미적분': 0.98312175,
 '국제경제': 0.97437143,
 '정보과학': 0.9697468,
 '경제수학': 0.97621167,
 '기하': 0.9814384,
 '지역이해': 0.95364356,
 '실용수학': 0.9810824,
 '심화수학Ⅰ': 0.98174924,
 '논리학': 0.9633277,
 '물리학Ⅱ': 0.9644685,
 '실용경제': 0.9747272,
 '수학과제탐구': 0.9823221,
 '사회문제탐구': 0.9842533,
 '생활과윤리': 0.96519107,
 '수학Ⅱ': 0.98224086,
 '교육학': 0.962739,
 '경제': 0.9732107,
 '철학': 0.9699824,
 '수학Ⅰ': 0.98243,
 '확률과통계': 0.98258054,
 '고급수학Ⅱ': 0.97937757,
 '세계문제와미래사회': 0.968441,
 '정보': 0.96782917,
 '국제법': 0.977346,
 '고급수학Ⅰ': 0.97329885,
 '논술': 0.970253,
 '심화수학Ⅱ': 0.9824661}

In [253]:
print(major_subject_sim)
print(job_major_sim)
#normalize similarity to 0.5~1
def normalize_sim(sim):
    max_sim = max(sim.values())
    min_sim = min(sim.values())
    for key, value in sim.items():
        sim[key] = (value-min_sim)/(max_sim-min_sim)*0.5+0.5
    return sim

# print(normalize_sim(job_major_sim))
# print(normalize_sim(major_subject_sim['통계학과']))

#rank


{'컴퓨터공학과': {'미적분': 0.97682333, '확률과통계': 0.973434, '물리학Ⅰ': 0.9658497, '화학Ⅰ': 0.9595792, '정보': 0.9655385, '실용수학': 0.9725909, '기하': 0.9758884, '수학과제탐구': 0.97299254, '물리학Ⅱ': 0.956769, '화학Ⅱ': 0.9589293, '생활과과학': 0.9680557, '공학일반': 0.9692286, '고급물리학': 0.96379966, '물리학실험': 0.96457493, '고급화학': 0.97029924, '화학실험': 0.9682761, '정보과학': 0.96203524, '융합과학탐구': 0.9703051, '과학과제연구': 0.96810985}, '컴퓨터과학과': {'미적분': 0.9780942, '확률과통계': 0.97392714, '물리학Ⅰ': 0.96719205, '화학Ⅰ': 0.962579, '정보': 0.96646965, '실용수학': 0.97304225, '기하': 0.97587013, '수학과제탐구': 0.97358036, '물리학Ⅱ': 0.9560193, '화학Ⅱ': 0.95386446, '생활과과학': 0.96731824, '공학일반': 0.9681281, '고급물리학': 0.9675238, '물리학실험': 0.96416587, '고급화학': 0.97070414, '화학실험': 0.96923494, '정보과학': 0.96178657, '융합과학탐구': 0.9711959, '과학과제연구': 0.9698388}, '컴퓨터디자인학과': {'미술': 0.95964235, '생활과윤리': 0.9490107, '윤리와사상': 0.9665637, '철학': 0.96232486, '심리학': 0.9680616, '교육학': 0.9474387, '미술창작': 0.9447296, '미술감상과비평': 0.9472014, '미술이론': 0.97798425, '미술사': 0.9780795, '드로잉': 0.974105, '미술전공실기': 

In [242]:
#dataframe from dictionary normalize_sim(major_subject_sim['통계학과']
tong = pd.DataFrame.from_dict((major_subject_sim['통계학과']), orient='index', columns=['similarity']).sort_values(by=['similarity'], ascending=False)
math = pd.DataFrame.from_dict((major_subject_sim['수학과']), orient='index', columns=['similarity'])

In [243]:
tong = tong.sort_values(by=['similarity'], ascending=False)
math = math.sort_values(by=['similarity'], ascending=False)

In [244]:
print(math)

        similarity
미적분       0.978782
수학과제탐구    0.978174
기하        0.978063
실용수학      0.977422
확률과통계     0.976900
수학Ⅰ       0.976513
수학Ⅱ       0.976219
심화수학Ⅰ     0.974895
심화수학Ⅱ     0.974095
고급수학Ⅱ     0.972537
경제수학      0.971773
논술        0.971712
실용경제      0.970645
경제        0.967802
정보        0.966161
철학        0.964538
정보과학      0.963439
고급수학Ⅰ     0.962108
논리학       0.961349
교육학       0.960427
물리학Ⅱ      0.957752
생활과윤리     0.950325


In [182]:
# major_subject all values to set
subject_set = set()
for key, value in major_subject.items():
    for v in value:
        subject_set.add(v)

subject_set

{'경제',
 '경제수학',
 '고급수학Ⅰ',
 '고급수학Ⅱ',
 '과학',
 '교육학',
 '국제경제',
 '국제관계와국제기구',
 '국제법',
 '금융일반',
 '기술가정',
 '기업자원통합관리',
 '기하',
 '논리학',
 '논술',
 '물리학Ⅱ',
 '미적분',
 '보험일반',
 '사회',
 '사회문제탐구',
 '사회문화',
 '사회탐구방법및사회과제연구',
 '생활ㆍ교양',
 '생활과윤리',
 '세계문제와미래사회',
 '수학',
 '수학Ⅰ',
 '수학Ⅱ',
 '수학과제탐구',
 '실용경제',
 '실용수학',
 '심화수학Ⅰ',
 '심화수학Ⅱ',
 '예산ㆍ자금',
 '정보',
 '정보과학',
 '정보처리와관리',
 '지역이해',
 '철학',
 '확률과통계'}

In [151]:
major_subject

{'수학과': ['수학',
  '사회',
  '정보',
  '과학',
  '수학Ⅰ',
  '수학Ⅱ',
  '미적분',
  '확률과 통계',
  '사회문화',
  '경제',
  '생활과 윤리',
  '정보',
  '논리학',
  '철학',
  '논술',
  '실용경제',
  '교육학',
  '실용수학',
  '기하',
  '경제수학',
  '수학과제탐구',
  '물리학Ⅱ',
  '심화수학Ⅰ',
  '심화수학Ⅱ',
  '고급수학Ⅰ',
  '고급수학Ⅱ',
  '정보과학',
  '정보처리와 관리'],
 '통계학과': ['수학',
  '사회',
  '기술가정',
  '생활ㆍ교양',
  '수학Ⅰ',
  '수학Ⅱ',
  '미적분',
  '확률과 통계',
  '경제',
  '사회문화',
  '생활과 윤리',
  '정보',
  '실용 경제',
  '실용 수학',
  '기하',
  '경제 수학',
  '수학 과제 탐구',
  '사회문제 탐구',
  '심화 수학Ⅰ',
  '정보과학',
  '국제 경제',
  '국제법',
  '지역 이해',
  '세계 문제와 미래 사회',
  '국제 관계와 국제기구',
  '사회 탐구 방법 및 사회과제 연구',
  '기업 자원 통합 관리',
  '금융 일반',
  '보험 일반',
  '예산ㆍ자금']}

In [173]:
df_job_major_subject[df_job_major_subject['job'] == "통계학연구원"].subject_details.values[9]

'정보처리와 관리'

In [175]:
df_subject_info[df_subject_info['subject_name'].str.contains('심화')].head()

,index,subject_name,description,ability,related_subjects,type
5,5,심화 국어,초·중·고 공통 과목 '국어'에서 학습한 결과를 바탕으로 하여 학습자의 학문 탐구 ...,"비판적·창의적 역량, 자료·정보·활용 역량, 의사소통 역량, 공동체·대인 관계 역량...","['문예 창작 입문', ' 문학 개론', ' 문장론', ' 고전문학 감상', ' 현...",common
79,0,심화 수학Ⅰ,"공통 과목인'수학'을 학습한 후에 선택할 수 있는 전문 교과 과목으로, 수학 일반 ...",,[],prof
80,1,심화 수학Ⅱ,전문 교과 과목인'심화 수학Ⅰ'을 학습한 후에 선택할 수 있는 전문 교과 과목으로 ...,,[],prof
104,25,체육 전공 실기 심화,체육 전공 실기 심화는 전공 실기의 훈련과 방법을 이해하고 기본 기술 및 응용 기술...,,[],prof
157,78,심화 영어 회화Ⅰ,일반적 주제에 관해 영어로 듣거나 말하는 능력을 기르는 과목으로'심화 영어 회화 I...,,[],prof
